In [1]:
import pandas as pd
import datetime
from scipy.fftpack import fft
import matplotlib.pyplot as plt
from matplotlib.pylab import mpl
import numpy as np
mpl.rcParams['font.sans-serif'] = ["times new roman"]
plt.rcParams['font.family'] = 'times new roman'
plt.rcParams['font.sans-serif'] = ["times new roman"]

%config InlineBackend.figure_format = 'retina'

In [2]:
def ShowWave(DataFilePath,EventFilePath,InIndex):
    df = pd.read_csv(DataFilePath)
    event_df = pd.read_csv(EventFilePath)
    EventTime = event_df.Time
    MidTime = datetime.datetime.strptime(str(EventTime[InIndex]),"%Y/%m/%d %H:%M:%S.%f")
    StartTime = MidTime - datetime.timedelta(seconds=1)
    EndTime = MidTime + datetime.timedelta(seconds=1)
    WaveList = [] # Waveform data

    TimePreDf = df[(df['RecvTime']<= MidTime.strftime("%Y/%m/%d %H:%M:%S.%f"))&(df['RecvTime']>= StartTime.strftime("%Y/%m/%d %H:%M:%S.%f"))].tail(8)

    for row in TimePreDf.itertuples():
        WaveList.append(row)
    TimeAfterDf = df[(df['RecvTime']<= EndTime.strftime("%Y/%m/%d %H:%M:%S.%f"))&(df['RecvTime']>= MidTime.strftime("%Y/%m/%d %H:%M:%S.%f"))].head(8)
    for row in TimeAfterDf.itertuples():
        WaveList.append(row)
    IList = []
    VList = []
    x = []
    Count = 0
    setp = 20/1024
    for row in WaveList:
        # print(row)
        if row.Channel == 1:
            VL = row.Value.split('|')
            for data in VL:
                VList.append(int(data)/12.85)
        if row.Channel == 2:
            IL = row.Value.split('|')
            for data in IL:
                IList.append(int(data)/32768.0 * 10 * 3)
                result = setp*Count
                x.append(result)
                Count = Count + 1
    return x,IList,VList


In [4]:
x,FanOneIList,FanOneVList = ShowWave('E:\Code\stm32\Research\TDHA\Fan_1stSpeed\Fan_1st-speed_1.csv','E:\Code\stm32\Research\TDHA\Fan_1stSpeed\Fan_1st-speed_sign_1.csv',0)
size = len(FanOneIList)
size

8192

In [5]:
data = FanOneIList
FanOneFFT_I=fft(data)
FanOneAbs_I=np.abs(FanOneFFT_I)
FanOneAngle_I=np.angle(FanOneFFT_I)

In [6]:
x,FanThreeIList,FanThreeVList = ShowWave('E:\Code\stm32\Research\TDHA\Fan_3rdSpeed\Fan_3rd-speed_1.csv','E:\Code\stm32\Research\TDHA\Fan_3rdSpeed\Fan_3rd-speed_sign_1.csv',0)

In [7]:
data = FanThreeIList
FanThreeFFT_I=fft(data)
FanThreeAbs_I=np.abs(FanThreeFFT_I)
FanThreeAngle_I=np.angle(FanThreeFFT_I)

In [8]:
tempList = FanThreeIList[0:235]
for i in range(235):
    FanThreeIList.insert(0,tempList[i])

In [10]:

fig, ax = plt.subplots(figsize=(16, 6),dpi=500)
plt.plot(x,FanOneIList[0:size],color='#22b166',label="Fan One Speed",linewidth=2)
plt.plot(x,FanThreeIList[0:size],color='#7974bb',label="Fan Three Speed",linewidth=2)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tick_params(labelsize=16)  #刻度字体大小13
# plt.title('Fan Current Wave',fontsize=24)
plt.ylabel('Current[A]',fontsize=24)
plt.xlabel('Time[ms]',fontsize=24)
plt.legend(['Fan One Speed','Fan Three Speed'])
plt.legend(loc=1)
plt.savefig('./FanCurrentWave.svg',bbox_inches='tight')

In [11]:
x = []
for i in range(size):
    x.append(i*6.25)


In [12]:

fig, ax = plt.subplots(figsize=(16, 6),dpi=500)
plt.plot(x[0:int(size/128)],FanOneAbs_I[0:int(size/128)],color='#22b166',label="Fan One Speed",linewidth=1)
plt.plot(x[0:int(size/128)],FanThreeAbs_I[0:int(size/128)],color='#7974bb',label="Fan Three Speed",linewidth=1)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tick_params(labelsize=16)  #刻度字体大小13
# plt.title('Fan Current Wave',fontsize=24)
plt.ylabel('Amplitute',fontsize=24)
plt.xlabel('Frequency [Hz]',fontsize=24)
plt.legend(['Fan One Speed','Fan Three Speed'])
plt.legend(loc=1)
plt.savefig('./FanCurrentFFTWave.svg',bbox_inches='tight')